In [0]:
import pandas as pd
import re
import string
import nltk
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
stopwords = nltk. corpus.stopwords.words('arabic')
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
nltk.download('wordnet')
pd.set_option('display.max_colwidth',100)
ps = nltk.ISRIStemmer()
wn = nltk.WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
Lyrics_df = pd.read_csv('arabicLyrics.csv')


In [0]:
Lyrics_df.head()

,songID,Singer,SongTitle,SongWriter,Composer,LyricsOrder,Lyrics,SingerNationality,SongDialect
0,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,2,اروح لاحبابي والاقي الفرح ساكن عينهم,Morocco,Meghribi
1,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,3,ابتسم لافراحهم وانا من الهم احترق,Morocco,Meghribi
2,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,4,واسأل جروحي من ترى حس بعذابي منهم,Morocco,Meghribi
3,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,5,وبالحقيقه انصدم محدن معه همي فرق,Morocco,Meghribi
4,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,6,دورت في كل الوجيه حسيت غربه بينهم,Morocco,Meghribi


In [1]:
#Cleaning

In [0]:

def clean_Lyrics(Lyrics):
    Lyrics = "".join([word.lower() for word in Lyrics if word not in string.punctuation])
    punctuation = ['،؛؟”0123456789“']  
    Lyrics = "".join([word.lower() for word in Lyrics if word not in punctuation])
    tokens = re.split('\W+', Lyrics)
    Lyrics = [word for word in tokens if word not in stopwords]
    regex = re.compile('[^ا-ي]')
    Lyrics = [regex.sub('', word) for word in Lyrics]
    Lyrics = [word for word in Lyrics if len(word) > 0]
    ignore = ['وانت','ربي','انه','الي','كله','كان','منك','ني','انت',
              'وين','اذا','او','عنك','منك','ونا','من','ماهو','فوق','كنت','وفي',
              'الا','وسط','فيك','انك','صار','مني','قبل','مو','ربي','شي','ما','اللي', 'للي', 'بلا'
              'وش','وكل','على','يالي','ماني','فيك','انتي','والله','مثل',]
    Lyrics = [word for word in Lyrics if word not in ignore]
    return Lyrics

In [0]:
Lyrics_df['Clean_Lyrics'] = Lyrics_df['Lyrics']. apply(lambda x: clean_Lyrics(x.lower())) 
Lyrics_df['SongDialect'] = Lyrics_df['SongDialect']. apply(lambda x: x.lower())
Lyrics_df['SingerNationality'] = Lyrics_df['SingerNationality']. apply(lambda x: x.lower())
                                                                 
Lyrics_df.head()

,songID,Singer,SongTitle,SongWriter,Composer,LyricsOrder,Lyrics,SingerNationality,SongDialect,Clean_Lyrics
0,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,2,اروح لاحبابي والاقي الفرح ساكن عينهم,morocco,meghribi,"[اروح, لاحبابي, والاقي, الفرح, ساكن, عينهم]"
1,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,3,ابتسم لافراحهم وانا من الهم احترق,morocco,meghribi,"[ابتسم, لافراحهم, وانا, الهم, احترق]"
2,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,4,واسأل جروحي من ترى حس بعذابي منهم,morocco,meghribi,"[واسل, جروحي, ترى, حس, بعذابي, منهم]"
3,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,5,وبالحقيقه انصدم محدن معه همي فرق,morocco,meghribi,"[وبالحقيقه, انصدم, محدن, معه, همي, فرق]"
4,1537,ابتسام,اروح لاحبابي,ملامح,بندر بن فهد,6,دورت في كل الوجيه حسيت غربه بينهم,morocco,meghribi,"[دورت, الوجيه, حسيت, غربه, بينهم]"


In [0]:
Lyrics_df.shape[0]

527896

### Using FARASA API for Lemmatization 

In [0]:
import json
import http.client

def lemma(Clean_Lyrics):
    
    text = ' '.join([word for word in Clean_Lyrics])
    #text = text.replace('"', '')
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    payload = "{\"text\":\"%s\"}"% text
    payload = payload.encode('utf-8')

    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }

    conn.request("POST", "/msa/webapi/lemma", payload, headers)

    res = conn.getresponse()

    data = res.read().decode('utf-8')
    #print(data)
    data_dict = json.loads(data)
    return data_dict['result']

In [0]:
Lyrics_df['Lyrics_lemmatized'] = Lyrics_df['Clean_Lyrics'].map(lambda x: lemma(x))             
Lyrics_df.head(10)

In [0]:
#Lyrics_lemma = pd.read_csv('Lyrics_lemma.csv')
start_count = 527499
end_count = 527896
while(end_count < Lyrics_df.shape[0]):
    Lyrics_df['Lyrics_lemmatized'].iloc[start_count:end_count] = Lyrics_df['Clean_Lyrics'].iloc[start_count:end_count].map(lambda x: lemma(x))
    Lyrics_df.to_csv('/content/drive/My Drive/ColabNotebooks/Lyrics_farasa.csv')
    print(end_count)
    start_count = end_count
    end_count = end_count + 397
    if end_count > Lyrics_df.shape[0]:
        end_count = Lyrics_df.shape[0]        
Lyrics_df.tail(397)

,songID,Singer,SongTitle,SongWriter,Composer,LyricsOrder,Lyrics,SingerNationality,SongDialect,Clean_Lyrics,Lyrics_lemmatized
527499,8710,يوسف الموصلي,بين سجن وبيت اشباح,غير معروف,غير معروف,21,فى الثورة بعد عصيان,sudan,sudan,"[فى, الثورة, عصيان]","[فى, ثورة, عصيان]"
527500,8710,يوسف الموصلي,بين سجن وبيت اشباح,غير معروف,غير معروف,22,يا ضراع الولد الجارح,sudan,sudan,"[ضراع, الولد, الجارح]","[ضراع, ولد, جارح]"
527501,8710,يوسف الموصلي,بين سجن وبيت اشباح,غير معروف,غير معروف,23,بسلاحك فى المليان,sudan,sudan,"[بسلاحك, فى, المليان]","[سلاح, فى, ملي]"
527502,8710,يوسف الموصلي,بين سجن وبيت اشباح,غير معروف,غير معروف,24,الكى للالم الناتح,sudan,sudan,"[الكى, للالم, الناتح]","[كى, الم, ناتح]"
527503,8710,يوسف الموصلي,بين سجن وبيت اشباح,غير معروف,غير معروف,25,والعبرة لكل غبيان,sudan,sudan,"[والعبرة, لكل, غبيان]","[عبرة, كل, غبي]"
...,...,...,...,...,...,...,...,...,...,...,...
527891,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,37,مالي وياهم ذنب,iraq,iraqi,"[مالي, وياهم, ذنب]","[مالي, يا, ذنب]"
527892,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,38,حرقت الروح لمن فارقتهم,iraq,iraqi,"[حرقت, الروح, لمن, فارقتهم]","[حرق, روح, من, فارق]"
527893,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,39,بجيت ومن دموعي غرقتهم,iraq,iraqi,"[بجيت, دموعي, غرقتهم]","[جيت, دمعة, غرق]"
527894,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,40,ايش قال القلب لمن ودعتهم,iraq,iraqi,"[ايش, قال, القلب, لمن, ودعتهم]","[ايش, قال, قلب, من, ودع]"


In [0]:
farasa =pd.read_csv('/content/drive/My Drive/ColabNotebooks/Lyrics_farasa.csv')

In [0]:
farasa.iloc[490498:527896,:]

,Unnamed: 0,songID,Singer,SongTitle,SongWriter,Composer,LyricsOrder,Lyrics,SingerNationality,SongDialect,Clean_Lyrics,Lyrics_lemmatized
490498,490498,23855,نوال الكويتية,مشتاق,تركى,ماجد المخيني,21,جروحي ما ارحم إجروحك,kuwait,gulf,"['جروحي', 'ارحم', 'جروحك']","['جروحي', 'ارحم', 'جروحك']"
490499,490499,23855,نوال الكويتية,مشتاق,تركى,ماجد المخيني,22,و إدموعي ما امسح دموعك,kuwait,gulf,"['و', 'دموعي', 'امسح', 'دموعك']","['و', 'دموعي', 'امسح', 'دموعك']"
490500,490500,23855,نوال الكويتية,مشتاق,تركى,ماجد المخيني,23,و خضوعي يوم انا طوعك,kuwait,gulf,"['و', 'خضوعي', 'يوم', 'انا', 'طوعك']","['و', 'خضوعي', 'يوم', 'انا', 'طوعك']"
490501,490501,23855,نوال الكويتية,مشتاق,تركى,ماجد المخيني,24,يوم ارجيك لا تخليني,kuwait,gulf,"['يوم', 'ارجيك', 'تخليني']","['يوم', 'ارجيك', 'تخليني']"
490502,490502,23855,نوال الكويتية,مشتاق,تركى,ماجد المخيني,25,لا اصدك كل ما نديت,kuwait,gulf,"['اصدك', 'نديت']","['اصدك', 'نديت']"
...,...,...,...,...,...,...,...,...,...,...,...,...
527891,527891,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,37,مالي وياهم ذنب,iraq,iraqi,"['مالي', 'وياهم', 'ذنب']","['مالي', 'يا', 'ذنب']"
527892,527892,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,38,حرقت الروح لمن فارقتهم,iraq,iraqi,"['حرقت', 'الروح', 'لمن', 'فارقتهم']","['حرق', 'روح', 'من', 'فارق']"
527893,527893,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,39,بجيت ومن دموعي غرقتهم,iraq,iraqi,"['بجيت', 'دموعي', 'غرقتهم']","['جيت', 'دمعة', 'غرق']"
527894,527894,1093,يونس الخطيب,احن الهم دويتو ادريس ويونس الخطيب,عبد الله العبد الله,يونس الخطيب,40,ايش قال القلب لمن ودعتهم,iraq,iraqi,"['ايش', 'قال', 'القلب', 'لمن', 'ودعتهم']","['ايش', 'قال', 'قلب', 'من', 'ودع']"
